## 2024 11 26 in-place operation bugs

In [13]:
import torch
import torch.nn as nn
print(torch.__version__)

2.2.2


### mimic nemo feature and tag

In [14]:
data = torch.randn((5,100,100))
feature = torch.randn((5,100,100))
data.requires_grad = True
feature.requires_grad = True

mlp1 = torch.randn((100,100))
mlp2 = torch.randn((100,100))

data.requires_grad = True
feature.requires_grad = True
mlp1.requires_grad = True
mlp2.requires_grad = True

x1 = data @ mlp1
x1.retain_grad()

x2 = feature @ mlp2
x2.retain_grad()

x1 += x2

loss = x1.sum()

In [15]:
loss.backward()

In [29]:
# set all gradients to zero
data.grad.zero_()
feature.grad.zero_()
mlp1.grad.zero_()
mlp2.grad.zero_()
# delete the computation graph
x1 = x1.detach()
x2 = x2.detach()
loss = loss.detach()


In [31]:
x1.requires_grad = True
x2.requires_grad = True
mlp1.requires_grad = True
mlp2.requires_grad = True

x1.retain_grad()
x2.retain_grad()
mlp1.retain_grad()
mlp2.retain_grad()

x1 = data @ mlp1
x1.retain_grad()

x2 = feature @ mlp2
x2.retain_grad()

x1 = x1 + x2
loss = x1.sum()

In [32]:
loss.backward()

In [33]:
x1.grad, x2.grad, mlp1.grad, mlp2.grad

/tmp/ipykernel_40792/733781882.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1711403380481/work/build/aten/src/ATen/core/TensorBody.h:489.)
  x1.grad, x2.grad, mlp1.grad, mlp2.grad


(None,
 tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
